# Customer Segmentation 

This notebook will focuss on the creation, deployment, 
monitoring and management of a machine learning model for performing 
customer segmentation. We will be using an e-commerce dataset detailing actual purchases made by  ∼ 4000 customers over a period of one year (from 2010/12/01 to 2011/12/09). 

In this notebook we will: 

- explore subset of dataset
- train several models on pre-processed dataset
- deploy trained models to Seldon 
- train an anchor tabular explainer and update Seldon deployment with explainer
- train an outlier detector (variational autoencoder) and drift detector (k-s tests) and update deployment 

### Prerequisites

In [1]:
!pip install seldon-deploy-sdk
!pip install alibi
!pip install alibi-detect
!pip install fsspec
!pip install gcsfs
!pip install dill

     |████████████████████████████████| 614kB 7.0MB/s 
     |████████████████████████████████| 204kB 14.4MB/s 
     |████████████████████████████████| 3.2MB 15.5MB/s 
     |████████████████████████████████| 317kB 8.7MB/s 
     |████████████████████████████████| 28.0MB 148kB/s 
  Created wheel for spacy-lookups-data: filename=spacy_lookups_data-0.1.0-py2.py3-none-any.whl size=28052145 sha256=31998e0cd9b65f5a6cdba13a97b2eefea6021016a8be9746e1a1d265ff692029
  Stored in directory: /root/.cache/pip/wheels/2a/2b/0a/d6fb6235c56d014d224bca760d15d7cbdd820813085ffcd35d
Successfully built spacy-lookups-data
     |████████████████████████████████| 153kB 7.6MB/s 
     |████████████████████████████████| 2.1MB 15.1MB/s 
     |████████████████████████████████| 3.3MB 32.1MB/s 
     |████████████████████████████████| 901kB 34.1MB/s 
     |████████████████████████████████| 112kB 7.2MB/s 
     |████████████████████████████████| 1.3MB 9.9MB/s 
     |████████████████████████████████| 296kB 31.5MB/s 
     |█

In [2]:
from sklearn import preprocessing, model_selection, metrics, feature_selection
from sklearn.svm import SVC
from sklearn import neighbors, linear_model, svm, tree, ensemble
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import confusion_matrix, f1_score
from sklearn.metrics import accuracy_score
import numpy as np
import pickle 
import joblib
import dill
import os 
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import pandas as pd
from tensorflow.keras.layers import Dense, InputLayer
from alibi.explainers import AnchorTabular
from alibi_detect.datasets import fetch_kdd
from alibi_detect.models.tensorflow.losses import elbo
from alibi_detect.od import OutlierVAE
from alibi_detect.cd import TabularDrift
from alibi_detect.utils.data import create_outlier_batch
from alibi_detect.utils.fetching import fetch_detector
from alibi_detect.utils.saving import save_detector, load_detector
from alibi_detect.utils.visualize import plot_instance_score, plot_feature_outlier_tabular, plot_roc
from seldon_deploy_sdk import Configuration, ApiClient, SeldonDeploymentsApi, OutlierDetectorApi, DriftDetectorApi
from seldon_deploy_sdk.auth import OIDCAuthenticator

Download datasets:

In [3]:
!gsutil -m cp -r gs://tom-seldon-examples/datareply-workshop/data .

Copying gs://tom-seldon-examples/datareply-workshop/data/X_test.npy...
Copying gs://tom-seldon-examples/datareply-workshop/data/X_train.npy...
Copying gs://tom-seldon-examples/datareply-workshop/data/Y_train.npy...
Copying gs://tom-seldon-examples/datareply-workshop/data/data.csv...
Copying gs://tom-seldon-examples/datareply-workshop/data/Y_test.npy...


### Data Exploration

Typically e-commerce datasets are proprietary and consequently hard to find among publicly available data. However, The UCI Machine Learning Repository has curated a retail dataset containing actual transactions from 2010 and 2011. The dataset is maintained on their site, where it can be found by the title "Online Retail". The following description is provided via Kaggle: 

"This is a transnational data set which contains all the transactions occurring between 01/12/2010 and 09/12/2011 for a UK-based and registered non-store online retail.The company mainly sells unique all-occasion gifts. Many customers of the company are wholesalers."

We will first explore the dataset and then train models on a pre-processed version of the dataset that was transformed through following the fantastic work of Fabien Daniel whose methods are detailed in this notebook: https://www.kaggle.com/fabiendaniel/customer-segmentation

Lets dive in and explore a subset of the dataset:

In [4]:
df_initial = pd.read_csv('data/data.csv', encoding="ISO-8859-1", nrows=3000,
                         dtype={'CustomerID': str,'InvoiceID': str})

print('Dataframe dimensions:', df_initial.shape)

Dataframe dimensions: (3000, 8)


In [5]:
df_initial['InvoiceDate'] = pd.to_datetime(df_initial['InvoiceDate'])

# gives some infos on columns types and numer of null values
tab_info=pd.DataFrame(df_initial.dtypes).T.rename(index={0:'column type'})
tab_info=tab_info.append(pd.DataFrame(df_initial.isnull().sum()).T.rename(index={0:'null values (nb)'}))
tab_info=tab_info.append(pd.DataFrame(df_initial.isnull().sum()/df_initial.shape[0]*100).T.
                         rename(index={0:'null values (%)'}))
display(tab_info)

# show first lines
display(df_initial[:5])

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
column type,object,object,object,int64,datetime64[ns],float64,object,object
null values (nb),0,0,10,0,0,0,1081,0
null values (%),0,0,0.333333,0,0,0,36.0333,0


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom


We have:

**InvoiceNo**: Invoice number. Nominal, a 6-digit integral number uniquely assigned to each transaction. If this code starts with letter 'c', it indicates a cancellation.

**StockCode**: Product (item) code. Nominal, a 5-digit integral number uniquely assigned to each distinct product.

**Description**: Product (item) name. Nominal.

**Quantity**: The quantities of each product (item) per transaction. Numeric.

**InvoiceDate**: Invice Date and time. Numeric, the day and time when each transaction was generated.

**UnitPrice**: Unit price. Numeric, Product price per unit in sterling.

**CustomerID**: Customer number. Nominal, a 5-digit integral number uniquely assigned to each customer.

**Country**: Country name. Nominal, the name of the country where each customer resides. 

We will be training models using a transformed version of this dataset. Rather than detailing each individual item bought, the transformed dataset details the spending behavoir of each individual customer across different categories of products. We don't have any product categories in the original dataset so we create this feature through analysing keywords used within product descriptions and applying k-means clustering to create a finite number of product clusters. We find in our analysis that 5 clusters is a suitable number to seperate products without creating clusters containing too few products (calculated using silhouette score). The following wordclouds demonstrate the splits we generate: 

<img src="https://github.com/ribenamaplesyrup/seldon-workshops/blob/main/download.png?raw=1">

We see that cluster no1 contains words we might associate more with gifts ('christmas', 'card', 'wrap', 'decoration') and no3 more with luxury goods ('lace', 'necklace'), although some of the other clusters are less differentiated.

Based on customer spending across each of these categories of product, we can similarly use k-means to cluster customers into different segments. Performing similar analysis to how we clustered product categories, we find 11 clusters is the most suitable for segmenting customers within this dataset.

In [6]:
columns = ['mean', 'categ_0', 'categ_1', 'categ_2', 'categ_3', 'categ_4' ] # change to feature_names?
class_names = ['0','1','2','3','4','5','6','7','8','9','10']
X_train = np.load('data/X_train.npy')
Y_train = np.load('data/Y_train.npy')
X_test = np.load('data/X_test.npy')
Y_test = np.load('data/Y_test.npy')


The following transformed features specific to each unique customer:

**mean**: mean spend across all transactions

**categ_0**: percentage of total spend on products within category 0

**categ_1**: percentage of total spend on products within category 1

**categ_2**: percentage of total spend on products within category 2

**categ_3**: percentage of total spend on products within category 3

**categ_4**: percentage of total spend on products within category 4

**customer_category**: integer signifying which of the 11 categories the customer aligns most with

In [ ]:
### high level summarise notebook in how we get from the original dataset to the transformed dataset with 
### product category, mean and customer category features.

- Mean refers to the average amount spent within each invoice. 
- Each category represents a different category of products and the value is the percentage of total spend within that category across all invoices. 

In [ ]:
columns = ['mean', 'categ_0', 'categ_1', 'categ_2', 'categ_3', 'categ_4' ] # change to feature_names?
class_names = ['0','1','2','3','4','5','6','7','8','9','10']
X_train = np.load('data/X_train.npy')
Y_train = np.load('data/Y_train.npy')
X_test = np.load('data/X_test.npy')
Y_test = np.load('data/Y_test.npy')

In [ ]:
X_train

In [ ]:
### figure out the max and min mean - useful for outliers 

## Model Training

In this section we will adjust a classifier that will classify consumers in the different client categories. 

### Logistic Regression

In [ ]:
lr = linear_model.LogisticRegression(max_iter=4000)
lr.fit(X_train, Y_train)

In [ ]:
accuracy_score(Y_test, lr.predict(X_test))

In [ ]:
### Add another trained model

In [ ]:
models = ['lr']

for model in models:
    if not os.path.exists('models/' + model):
        os.makedirs('models/' + model)

In [ ]:
filename = 'models/lr/model.joblib'
joblib.dump(lr, filename)

### Push model artefacts to GCP

To push models to GCP you will create a unique folder with your name. Uncomment the command and replace <YOUR NAME> with your name.

In [ ]:
# Push artefact to GCP
# !gsutil cp model/<.sav model file> gs://tom-seldon-examples/datareply-workshop/models/<YOUR NAME>/<MODEL TYPE>/model.joblib
!gsutil cp models/lr/model.joblib gs://tom-seldon-examples/datareply-workshop/models/sgreaves/lr/model.joblib

### Deploy models to Seldon

We can now deploy our model to the dedicated Seldon Deploy cluster which we have configured for this workshop. To do so we will interact with the Seldon Deploy SDK and deploy our model using that.

First, setting up the configuration and authentication required to access the cluster. Make sure to fill in the SD_IP variable to be the same as the cluster you are using.

In [ ]:
SD_IP = "139.59.203.129"

config = Configuration()
config.host = f"http://{SD_IP}/seldon-deploy/api/v1alpha1"
config.oidc_client_id = "sd-api"
config.oidc_server = f"http://{SD_IP}/auth/realms/deploy-realm"

def auth():
    auth = OIDCAuthenticator(config)
    config.access_token = auth.authenticate("admin@seldon.io", "12341234")
    api_client = ApiClient(config)
    return api_client

In [ ]:
### edit the following explanation ###

Now we have configured the IP correctly as well as setup our authentication function we can desrcibe the deployment we would like to create.

You will need to fill in the DEPLOYMENT_NAME, NAMESPACE, and the MODEL_LOCATION, the rest of the deployment description has been templated for you.

For the MODEL_LOCATION you do not need to specify the path all the way up to model.bst e.g. if you saved your classifier under gs://tom-seldon-examples/my-workshop/tom/model.bst your MODEL_LOCATION should be gs://tom-seldon-examples/my-workshop/tom and Seldon will automatically pick up the classifier artifact stored there.

You will need to create a unique deployment name. A good example format would be <YOUR NAME> + <MODEL> so "sgreaveslogreg" in my case (be careful not to use any upper-case letters or other characters like "_". We will also need to specify our model_location which needs to be the folder we pushed our model.joblib file to.

In [ ]:
DEPLOYMENT_NAME = "sgreaveslogreg"
MODEL_LOCATION = "gs://tom-seldon-examples/datareply-workshop/models/sgreaves/lr"

In [ ]:
NAMESPACE = "test"

PREPACKAGED_SERVER = "SKLEARN_SERVER"

CPU_REQUESTS = "1"
MEMORY_REQUESTS = "1Gi"

CPU_LIMITS = "1"
MEMORY_LIMITS = "1Gi"

mldeployment = {
    "kind": "SeldonDeployment",
    "metadata": {
        "name": DEPLOYMENT_NAME,
        "namespace": NAMESPACE,
        "labels": {
            "fluentd": "true"
        }
    },
    "apiVersion": "machinelearning.seldon.io/v1alpha2",
    "spec": {
        "name": DEPLOYMENT_NAME,
        "annotations": {
            "seldon.io/engine-seldon-log-messages-externally": "true"
        },
        "protocol": "seldon",
        "transport": "rest",
        "predictors": [
            {
                "componentSpecs": [
                    {
                        "spec": {
                            "containers": [
                                {
                                    "name": f"{DEPLOYMENT_NAME}-container",
                                    "resources": {
                                        "requests": {
                                            "cpu": CPU_REQUESTS,
                                            "memory": MEMORY_REQUESTS
                                        },
                                        "limits": {
                                            "cpu": CPU_LIMITS,
                                            "memory": MEMORY_LIMITS
                                        }
                                    }
                                }
                            ]
                        }
                    }
                ],
                "name": "default",
                "replicas": 1,
                "traffic": 100,
                "graph": {
                    "implementation": PREPACKAGED_SERVER,
                    "modelUri": MODEL_LOCATION,
                    "name": f"{DEPLOYMENT_NAME}-container",
                    "endpoint": {
                        "type": "REST"
                    },
                    "parameters": [],
                    "children": [],
                    "logger": {
                        "mode": "all"
                    }
                }
            }
        ]
    },
    "status": {}
}

We can now invoke the `SeldonDeploymentsApi` and create a new Seldon Deployment. 

Time for you to get your hands dirty. You will use the Seldon Deploy SDK to create a new Seldon deployment. You can find the reference documentation [here](https://github.com/SeldonIO/seldon-deploy-sdk/blob/master/python/README.md). 

In [ ]:
deployment_api = SeldonDeploymentsApi(auth())
deployment_api.create_seldon_deployment(namespace=NAMESPACE, mldeployment=mldeployment)

Our model is now running as a fully fledged microservice. We can test it by sending a request: 

In [ ]:
# test model with this request: 
{"data": {"ndarray": [[169.44666667,19.82924814,22.28823229, 28.95207932,23.49411811,5.43632215]]}}

## Explainer

Next, we shall train an explainer to glean deeper insights into the decisions being made by our model. 

We will make use of the Anchors algorithm, which has a [production grade implementation available](https://docs.seldon.io/projects/alibi/en/stable/methods/Anchors.html) using the Seldon Alibi Explain library. 

The first step will be to write a simple prediction function which the explainer can call in order to query our logistic regression model.

In [ ]:
clf = lr

In [ ]:
predict_fn = lambda x: clf.predict(x)

We then initialise our Anchor explainer, using the AnchorTabular flavour provided by Alibi due to our data modality.

In [ ]:
explainer = AnchorTabular(predict_fn, columns)

In [ ]:
# Dataset is imbalanced - is this a problem?
# unique, counts = np.unique(Y_test, return_counts=True)
# print(np.asarray((unique, counts)).T)

In [ ]:
explainer.fit(X_train, disc_perc=(25, 50, 75))

We can now test our explainer by generating a prediction.

In [ ]:
idx = 0
print('Prediction: ', class_names[explainer.predictor(X_test[idx].reshape(1, -1))[0]])

In [ ]:
## explain precision and coverage 

In [ ]:
explanation = explainer.explain(X_test[idx], threshold=0.9)
print('Anchor: %s' % (' AND '.join(explanation.anchor)))
print('Precision: %.2f' % explanation.precision)
print('Coverage: %.2f' % explanation.coverage)

In [ ]:
## explain what dill is

In [ ]:
with open("models/lr/explainer.dill", "wb") as model_f:
        dill.dump(explainer, model_f)

In [ ]:
### explainers are quite large files, instead of pushing to GCP we'll skip that step and deploy a trained 
### explainer already in storage 

## Deployment

We can now deploy our explainer alongside our model. First we define the explainer configuration. 

In [ ]:
### train an explainer on the new dataset in a Python 3.6.1 environment then push to GCP and test deployment.

In [ ]:
EXPLAINER_TYPE = "AnchorTabular"
EXPLAINER_URI = "gs://tom-seldon-examples/datareply-workshop/pretrained/lr"

explainer_spec = {
                    "type": EXPLAINER_TYPE,
                    "modelUri": EXPLAINER_URI,
                    "containerSpec": {
                        "name": "",
                        "resources": {}
                    }
                }

In [ ]:
mldeployment['spec']['predictors'][0]['explainer'] = explainer_spec
mldeployment

In [ ]:
deployment_api = SeldonDeploymentsApi(auth())
deployment_api.create_seldon_deployment(namespace=NAMESPACE, mldeployment=mldeployment)

## Outlier Detection 


In [ ]:
### Provide explanations on all decisions made here. 

## Method

The Variational Auto-Encoder ([VAE](https://arxiv.org/abs/1312.6114)) outlier detector is first trained on a batch of unlabeled, but normal (*inlier*) data. Unsupervised training is desireable since labeled data is often scarce. The VAE detector tries to reconstruct the input it receives. If the input data cannot be reconstructed well, the reconstruction error is high and the data can be flagged as an outlier. The reconstruction error is either measured as the mean squared error (MSE) between the input and the reconstructed instance or as the probability that both the input and the reconstructed instance are generated by the same process.

In [ ]:
normal_batch = create_outlier_batch(X_train, Y_train, n_samples=2886, perc_outlier=0)
X_train, y_train = normal_batch.data.astype('float'), normal_batch.target
print(X_train.shape, y_train.shape)
print('{}% outliers'.format(100 * y_train.mean()))

In [ ]:
mean, stdev = X_train.mean(axis=0), X_train.std(axis=0)

In [ ]:
X_train = (X_train - mean) / stdev

## Load or define outlier detector

The pretrained outlier and adversarial detectors used in the example notebooks can be found [here](https://console.cloud.google.com/storage/browser/seldon-models/alibi-detect). You can use the built-in ```fetch_detector``` function which saves the pre-trained models in a local directory ```filepath``` and loads the detector. Alternatively, you can train a detector from scratch:

In [ ]:
load_outlier_detector = False

In [ ]:
filepath = '/content/models/OutlierVAE'
if not os.path.exists(filepath):
  os.mkdir(filepath)

In [ ]:
# define model, initialize, train and save outlier detector
    
n_features = X_train.shape[1]
latent_dim = 2
    
encoder_net = tf.keras.Sequential(
    [
     InputLayer(input_shape=(n_features,)),
     Dense(20, activation=tf.nn.relu),
     Dense(15, activation=tf.nn.relu),
     Dense(7, activation=tf.nn.relu)
     ])

decoder_net = tf.keras.Sequential(
    [
     InputLayer(input_shape=(latent_dim,)),
     Dense(7, activation=tf.nn.relu),
     Dense(15, activation=tf.nn.relu),
     Dense(20, activation=tf.nn.relu),
     Dense(n_features, activation=None)
     ])
    
# initialize outlier detector
od = OutlierVAE(threshold=None,  # threshold for outlier score
                score_type='mse',  # use MSE of reconstruction error for outlier detection
                encoder_net=encoder_net,  # can also pass VAE model instead
                decoder_net=decoder_net,  # of separate encoder and decoder
                latent_dim=latent_dim,
                samples=5)
# train
od.fit(X_train,
       loss_fn=elbo,
       cov_elbo=dict(sim=.01),
       epochs=30,
       verbose=True)

# save the trained outlier detector
save_detector(od, filepath)

The warning tells us we still need to set the outlier threshold. This can be done with the `infer_threshold` method. We need to pass a batch of instances and specify what percentage of those we consider to be normal via `threshold_perc`. Let's assume we have some data which we know contains around 5% outliers. The percentage of outliers can be set with `perc_outlier` in the `create_outlier_batch` function.

In [ ]:
np.random.seed(0)
perc_outlier = 5
threshold_batch = create_outlier_batch(X_train, Y_train, n_samples=100, perc_outlier=perc_outlier)
X_threshold, y_threshold = threshold_batch.data.astype('float'), threshold_batch.target
X_threshold = (X_threshold - mean) / stdev
print('{}% outliers'.format(100 * y_threshold.mean()))

In [ ]:
od.infer_threshold(X_threshold, threshold_perc=100-perc_outlier)
print('New threshold: {}'.format(od.threshold))

In [ ]:
save_detector(od, filepath)

## Detect outliers

We now generate a batch of data with 10% outliers and detect the outliers in the batch. 


In [ ]:
### provide more explanation here of what we're doing and why

In [ ]:
np.random.seed(1)
outlier_batch = create_outlier_batch(X_train, Y_train, n_samples=2886, perc_outlier=10)
X_outlier, y_outlier = outlier_batch.data.astype('float'), outlier_batch.target
X_outlier = (X_outlier - mean) / stdev
print(X_outlier.shape, y_outlier.shape)
print('{}% outliers'.format(100 * y_outlier.mean()))

Predict outliers:

In [ ]:
od_preds = od.predict(X_outlier,
                      outlier_type='instance',    # use 'feature' or 'instance' level
                      return_feature_score=True,  # scores used to determine outliers
                      return_instance_score=True)
print(list(od_preds['data'].keys()))

## Display results

In [ ]:
labels = outlier_batch.target_names
y_pred = od_preds['data']['is_outlier']
f1 = f1_score(y_outlier, y_pred)
print('F1 score: {:.4f}'.format(f1))
cm = confusion_matrix(y_outlier, y_pred)
df_cm = pd.DataFrame(cm, index=labels, columns=labels)
sns.heatmap(df_cm, annot=True, cbar=True, linewidths=.5)
plt.show()

In [ ]:
plot_instance_score(od_preds, y_outlier, labels, od.threshold)

In [ ]:
roc_data = {'VAE': {'scores': od_preds['data']['instance_score'], 'labels': y_outlier}}
plot_roc(roc_data)

## Investigate instance level outlier

We can now take a closer look at some of the individual predictions on `X_outlier`. 

In [ ]:
X_recon = od.vae(X_outlier).numpy()  # reconstructed instances by the VAE

In [ ]:
plot_feature_outlier_tabular(od_preds,
                             X_outlier,
                             X_recon=X_recon,
                             threshold=od.threshold,
                             instance_ids=None,  # pass a list with indices of instances to display
                             max_instances=5,  # max nb of instances to display
                             top_n=5,  # only show top_n features ordered by outlier score
                             outliers_only=False,  # only show outlier predictions
                             feature_names=columns,  # add feature names
                             figsize=(20, 30))

In [ ]:
### provide a tangiable explained example of an obvious outlier like some large or small mean coupled with high spend in 
### a single category.

In [ ]:
### Push to GCP and then DEPLOY

## Drift Detection

### Method

The drift detector applies feature-wise two-sample [Kolmogorov-Smirnov](https://en.wikipedia.org/wiki/Kolmogorov%E2%80%93Smirnov_test) (K-S) tests for the continuous numerical features.

We split the data in a reference set and 2 test sets on which we test the data drift:

In [ ]:
n_ref = 900
n_test = 900

X_ref, X_t0, X_t1 = X_train[:n_ref], X_train[n_ref:n_ref + n_test], X_train[n_ref + n_test:n_ref + 2 * n_test]
X_ref.shape, X_t0.shape, X_t1.shape

### Detect drift

Initialize the detector:

In [ ]:
# Where are we specifying batch size - does this play a part in the training?

In [ ]:
cd = TabularDrift(X_ref, p_val=.05)

In [ ]:
preds = cd.predict(X_t0)
labels = ['No!', 'Yes!']
print('Drift? {}'.format(labels[preds['data']['is_drift']]))

In [ ]:
for f in range(cd.n_features):
    stat = 'K-S'
    fname = columns[f]
    stat_val, p_val = preds['data']['distance'][f], preds['data']['p_val'][f]
    print(f'{fname} -- {stat} {stat_val:.3f} -- p-value {p_val:.3f}')

None of the feature-level p-values are below the threshold.

In [ ]:
preds['data']['threshold']

If you are interested in individual feature-wise drift, this is also possible:

In [ ]:
fpreds = cd.predict(X_t0, drift_type='feature')

In [ ]:
for f in range(cd.n_features):
    stat = 'K-S'
    fname = columns[f]
    is_drift = fpreds['data']['is_drift'][f]
    stat_val, p_val = fpreds['data']['distance'][f], fpreds['data']['p_val'][f]
    print(f'{fname} -- Drift? {labels[is_drift]} -- {stat} {stat_val:.3f} -- p-value {p_val:.3f}')

In [ ]:
preds = cd.predict(X_t1)
labels = ['No!', 'Yes!']
print('Drift? {}'.format(labels[preds['data']['is_drift']]))

We can again investigate the individual features:

In [ ]:
for f in range(cd.n_features):
    stat = 'K-S'
    fname = columns[f]
    is_drift = (preds['data']['p_val'][f] < preds['data']['threshold']).astype(int)
    stat_val, p_val = preds['data']['distance'][f], preds['data']['p_val'][f]
    print(f'{fname} -- Drift? {labels[is_drift]} -- {stat} {stat_val:.3f} -- p-value {p_val:.3f}')

In [ ]:
### add a tangiable drift example! 

In [ ]:
filepath = 'models/DriftDetector'
if not os.path.exists(filepath):
  os.mkdir(filepath)

save_detector(cd, filepath)

In [ ]:
### Push to GCP and Deploy 
!gsutil cp -r models/DriftDetector gs://tom-seldon-examples/datareply-workshop/models/sgreaves/lr/drift

In [ ]:
DD_URI = 'gs://tom-seldon-examples/datareply-workshop/models/sgreaves/lr/drift'
DD_NAME = 'KSDrift-Detector'

dd_config = {'deployment': DEPLOYMENT_NAME,
             'deployment_namespace': None,
             'namespace': 'seldon-logs',
             'params': {'drift_batch_size': '10',
                        'env_secret_ref': None,
                        'event_source': f'io.seldon.serving.dev-seldondeployment-{DEPLOYMENT_NAME}-drift',
                        'event_type': 'io.seldon.serving.inference.drift',
                        'http_port': '8080',
                        'model_name': DD_NAME,
                        'protocol': 'seldon.http',
                        'reply_url': 'http://seldon-request-logger.seldon-logs',
                        'storage_uri': DD_URI,
                        'user_permission': None},
             'prom_scraping': None,
             'url': None}

In [ ]:
dd_api = DriftDetectorApi(auth())
dd_api.create_drift_detector_seldon_deployment(name=DEPLOYMENT_NAME,
                                               namespace="dev",
                                               drift_detector=dd_config)